In [1]:
from prerootvertex import *
import itertools

3968*945**k

### Trying to transfer M7 from larger pre root trees to smaller
Firstly choose `mod_from` and `mod_to`.
Then iterate delta from 1 increasingly.
$f-subforest$  size is $$7(k - 1) + mod\_from$$
$t-subforest$  size is $$7(k - delta) + mod\_to$$
Try to transfer $M_{7}$.
$$\{f \cup M_7, t\} \rightarrow \{f, t \cup M_7\}$$
Criterion of success transfer:
$$\frac{i_{-}(f)}{i_{+}(f)} * \frac{i_{+}(t)}{i_{-}(t)} \geq 1$$

In [8]:
for mod_from in range(7):
    from_tree = PreRootVertex(mod_from, -1) #-1 because we check transfer from_tree without M7
    for mod_to in range(7):
        delta = 1
        for d in itertools.count(1):
            to_tree = PreRootVertex(mod_to, -d)

            assert (from_tree.excl_ind_cnt * to_tree.incl_ind_cnt).power_bases == (from_tree.incl_ind_cnt * to_tree.excl_ind_cnt).power_bases
            criterion = (from_tree.excl_ind_cnt * to_tree.incl_ind_cnt).substitute(0) / (from_tree.incl_ind_cnt * to_tree.excl_ind_cnt).substitute(0)
            if criterion >= 1:
                str = "mods: {} -> {}; delta: {}; criterion: {}".format(mod_from, mod_to, d, criterion)
                print(str)
                break

mods: 0 -> 0; delta: 1; criterion: 1
mods: 0 -> 1; delta: 1; criterion: 157565625/88529281
mods: 0 -> 2; delta: 1; criterion: 105/97
mods: 0 -> 3; delta: 1; criterion: 16544390625/8587340257
mods: 0 -> 4; delta: 1; criterion: 11025/9409
mods: 0 -> 5; delta: 1; criterion: 1737161015625/832972004929
mods: 0 -> 6; delta: 1; criterion: 1157625/912673
mods: 1 -> 0; delta: 4; criterion: 88529281/72335025
mods: 1 -> 1; delta: 1; criterion: 1
mods: 1 -> 2; delta: 3; criterion: 912673/893025
mods: 1 -> 3; delta: 1; criterion: 105/97
mods: 1 -> 4; delta: 3; criterion: 9409/8505
mods: 1 -> 5; delta: 1; criterion: 11025/9409
mods: 1 -> 6; delta: 3; criterion: 97/81
mods: 2 -> 0; delta: 2; criterion: 97/81
mods: 2 -> 1; delta: 1; criterion: 1500625/912673
mods: 2 -> 2; delta: 1; criterion: 1
mods: 2 -> 3; delta: 1; criterion: 157565625/88529281
mods: 2 -> 4; delta: 1; criterion: 105/97
mods: 2 -> 5; delta: 1; criterion: 16544390625/8587340257
mods: 2 -> 6; delta: 1; criterion: 11025/9409
mods: 3 ->

Max $delta$ value: 4
After applying all possible transfers:
$$\max_{f \text{ in root subtrees}} {\lfloor\frac{size_{-}(f)}{7}\rfloor} -\min_{t \text{ in root subtrees}} {\lfloor\frac{size_{-}(t)}{7}\rfloor} \leq 2$$
or:
$$\exists k \geq 23\; \forall f \text{ in root subtrees} \quad \lfloor\frac{size_{-}(f)}{7}\rfloor \in [k, k + 2]$$

Let's fix this *k*.
Let $$PRS = \{PreRootVertex(mod, delta) \;|\; mod \in [0, 7),\; delta \in [0, 2]\}$$
We will build universal swap graph *G*, where vertices are pairs of each two possible root subtrees and $G$ contains edge $(f, t)$ with value $c$, if there is exists universal swap from $f$ to $t$ for all $k \geq c$
$$V(G) = PRS \times PRS \\
E(G) \subset V(G) \times V(G) \times \mathbb{N}$$
The first element of pair is less than seven: $size_{-}(f)-7k < 7$

In [9]:
class Edge:
    def __init__(self, f: tuple, t: tuple, c, f12, f21_excl):
        self.f = f
        self.t = t
        self.c = c
        self.f12 = f12
        self.f21_excl = f21_excl

from collections import defaultdict

graph = [dict(), dict(), dict()]

max_delta = 2
max_dif = max_delta * 7 + 6

f_sz = [0, 0]
t_sz = [0, 0]
max_since = MIN_K_VAL
for f_sz[0] in range(7):
    for f_sz[1] in range(f_sz[0], max_dif + 1):
        from_forest = [PreRootVertex(f_sz[0] % 7, f_sz[0] // 7), PreRootVertex(f_sz[1] % 7, f_sz[1] // 7)]
        graph[2][tuple(f_sz)] = []
        cur_since = max_since
        for t_sz[0] in range(max_dif):
            transfer = t_sz[0] - f_sz[0]
            t_sz[1] = f_sz[1] - transfer
            if t_sz[1] < t_sz[0] or t_sz[1] > max_dif:
                continue
            to_forest = [PreRootVertex(t_sz[0] % 7, t_sz[0] // 7), PreRootVertex(t_sz[1] % 7, t_sz[1] // 7)]
            can_swap, since, F12, F21_excl = is_swap_edge(from_forest, to_forest)
            if can_swap:
                graph[2][tuple(f_sz)].append(Edge(tuple(f_sz), tuple(t_sz), since, F12, F21_excl))
                cur_since = min(cur_since, since)
        max_since = max(max_since, cur_since)

max_since

23

All swaps are possible if $k \geq 23$.
Lets print the list of possible subtrees for trees with two subtrees.

In [10]:
ans = [set(), set(), set()]
for f_sz[0] in range(7):
    for f_sz[1] in range(f_sz[0], max_dif + 1):
        if len(graph[2][tuple(f_sz)]) == 0:
            ans[2].add(tuple(f_sz))
            # print(bests[-1])
print("allowed pairs count of pre-root subtrees: {}".format(len(ans[2])))
print(*sorted(list(ans[2]), key=sum), sep='\n')

allowed pairs count of pre-root subtrees: 17
(0, 0)
(0, 1)
(0, 2)
(1, 2)
(2, 2)
(1, 4)
(2, 4)
(0, 7)
(4, 4)
(0, 9)
(4, 6)
(0, 11)
(6, 6)
(0, 13)
(0, 15)
(2, 15)
(2, 17)


Let's form the allowed tuples of **three** subtrees using information for two subtrees and same technique. Moreover, we can use the same technique for any count of pre-root subtrees.
We will stop when the count of different possible subtrees is less than three

In [11]:
def is_possible_forest(tup, prev_tups):
    for i in range(len(tup)):
        new_tup = tup[:i] + tup[i + 1:]
        min_elem = min(new_tup)
        while min_elem > 6:
            new_tup = tuple(el - 7 for el in new_tup)
            min_elem -= 7
        if new_tup not in prev_tups:
            return False
    return True

dif_set = set()

for subtree_cnt in itertools.count(start=3):
    graph.append(dict())
    dif_set.clear()
    for prev_from in ans[subtree_cnt - 1]:
        for last_from in range(max(prev_from), max_dif + 1):
            from_sizes = prev_from + (last_from, )
            if is_possible_forest(from_sizes, ans[subtree_cnt - 1]):
                graph[subtree_cnt][from_sizes] = []

    for from_sizes in graph[subtree_cnt].keys():
        from_sz = 0
        for sz in from_sizes:
            from_sz += sz
        from_forest = [PreRootVertex(sz % 7, sz // 7) for sz in from_sizes]
        cur_since = max_since
        for prev_to in ans[subtree_cnt - 1]:
            prev_to_sz = 0
            for sz in prev_to:
                prev_to_sz += sz
            last_to = from_sz - prev_to_sz
            if last_to < max(prev_to): # subtree tuple is ordered
                continue
            to_sizes = prev_to + (last_to, )

            to_forest = [PreRootVertex(sz % 7, sz // 7) for sz in to_sizes]
            can_swap, since, F12, F21_excl = is_swap_edge(from_forest, to_forest)
            if can_swap:
                graph[subtree_cnt][from_sizes].append(Edge(from_sizes, to_sizes, since, F12, F21_excl))
                cur_since = min(cur_since, since)
        max_since = max(max_since, cur_since)

    ans.append(set())
    for forest, edge_list in graph[subtree_cnt].items():
        if len(edge_list) == 0:
            ans[subtree_cnt].add(forest)
            dif_set.update(forest)

    print("subtree_cnt is {}, size is {}".format(subtree_cnt, len(ans[subtree_cnt])))
    # print(*sorted(list(ans[subtree_cnt])), sep='\n')
    if subtree_cnt == 8:
        break

subtree_cnt is 3, size is 24
subtree_cnt is 4, size is 31
subtree_cnt is 5, size is 38
subtree_cnt is 6, size is 45
subtree_cnt is 7, size is 52
subtree_cnt is 8, size is 59


In [12]:
dif_set

{0, 1, 2, 7, 9, 11, 13, 15}

In [13]:
max_since

23

In [14]:
ans[8]

{(0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 1),
 (0, 0, 0, 0, 0, 0, 0, 2),
 (0, 0, 0, 0, 0, 0, 0, 7),
 (0, 0, 0, 0, 0, 0, 0, 9),
 (0, 0, 0, 0, 0, 0, 0, 11),
 (0, 0, 0, 0, 0, 0, 0, 13),
 (0, 0, 0, 0, 0, 0, 0, 15),
 (0, 0, 0, 0, 0, 0, 1, 2),
 (0, 0, 0, 0, 0, 0, 2, 2),
 (0, 0, 0, 0, 0, 0, 2, 15),
 (0, 0, 0, 0, 0, 0, 7, 7),
 (0, 0, 0, 0, 0, 0, 7, 9),
 (0, 0, 0, 0, 0, 0, 9, 9),
 (0, 0, 0, 0, 0, 0, 9, 11),
 (0, 0, 0, 0, 0, 0, 11, 11),
 (0, 0, 0, 0, 0, 0, 11, 13),
 (0, 0, 0, 0, 0, 0, 13, 13),
 (0, 0, 0, 0, 0, 1, 2, 2),
 (0, 0, 0, 0, 0, 2, 2, 2),
 (0, 0, 0, 0, 0, 2, 2, 15),
 (0, 0, 0, 0, 0, 7, 7, 7),
 (0, 0, 0, 0, 0, 7, 7, 9),
 (0, 0, 0, 0, 0, 7, 9, 9),
 (0, 0, 0, 0, 0, 9, 9, 9),
 (0, 0, 0, 0, 0, 9, 9, 11),
 (0, 0, 0, 0, 0, 9, 11, 11),
 (0, 0, 0, 0, 0, 11, 11, 11),
 (0, 0, 0, 0, 2, 2, 2, 2),
 (0, 0, 0, 0, 7, 7, 7, 7),
 (0, 0, 0, 0, 7, 7, 7, 9),
 (0, 0, 0, 0, 7, 7, 9, 9),
 (0, 0, 0, 0, 7, 9, 9, 9),
 (0, 0, 0, 0, 9, 9, 9, 9),
 (0, 0, 0, 0, 9, 9, 9, 11),
 (0, 0, 0, 0, 9, 9, 11, 11),
 (0, 0,